In [ ]:
## T-REX    API :::: https://github.com/trexminer/T-Rex/wiki/API
'''
This program for get t-rex miner status from web api then save to InfluxDB and visualize with Grafana Dashboard 
'''

In [ ]:
import requests
import json

In [ ]:
import os
import sys, getopt, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [ ]:
api_version = 'test1'

In [ ]:
#Rig config parameters ( ip addresses and password)
ips =['192.168.0.1','192.168.0.2','192.168.0.3','192.168.0.4'']
password_='rig_password'

In [ ]:
#influxDB config parameters
url='http://influxDB:8086'
token = "influxDB access token"
org = "myOrg"
bucket = "myBucket"

In [ ]:
class ethMiner:
       
    gpusMeasurement={}
    gpusTags={}
    gpusFields={}
    
    
    def __init__(self,ip_,pass_):
        self.ip = ip_
        self.password = pass_        
        self.login={}
        self.summary={}

        self.workerMeasurement={}    
        self.workerTags ={'ip':ip_}
        self.workerFields={}
    
        self.gpus=[] 
        
    def login_(self):
        url_='http://' + self.ip + ':4067/login?password=' + self.password
        req = requests.get(url_)
        self.login= req.json()

    def logout_(self):
        url_='http://' + self.ip + ':4067/logout?sid=' + self.login['sid']
        req = requests.get(url_)
        self.login= req.json()
        
    def getSummary(self):
        try:
            #print ('getSummary:' +self.ip)
            url2= 'http://' + self.ip + ':4067/summary?sid=' + self.login['sid']    
            req= requests.get(url2)
            self.summary = req.json()
        except MaxRetryError as m:
            self.login_()
        except Exception as e:
            print('connection error!!')
            
            
    
    def resetWorkerField(self):
        try:
            if(self.workerMeasurement['fields']):
                f = self.workerMeasurement['fields']
                for key in f:
                    #print (key +' '+ str(type(f[key])))
                    if type(f[key]) == int:
                        f[key] = 0
                    elif type(f[key]) == float:
                        f[key] =0.00
                    elif type(f[key]) == str:
                        f[key] =''
        except Exception as e:
            print(e)
           
    def reset_(self):
        self.resetWorkerField()
        self.gpus=[] 
        
    def addWorkerField(self,key_,value_):
        tag_field=['url','user','worker','algorithm','description','driver','name','os','version'] 
            
        if(key_ in tag_field):
            self.workerTags[str(key_).replace(' ','_')]=str(value_)
        else:
            self.workerFields[str(key_).replace(' ','_')]=value_            
        

               
        return ''
    
    def extractWorker(self,data):
        #print('flatJson' )
        skipFields= ['gpus','watchdog_stat']
        
        for key,value in data.items():    
            if key not in skipFields:
                if isinstance(value, dict):
                    self.extractWorker(value)
                elif isinstance(value, list):
                    for val in value:
                        if isinstance(val, str):
                            #print (str(key)+'->'+str(value))
                            pass
                        elif isinstance(val, list):
                            pass
                        else:
                            self.extractWorker(val)
                else:
                    self.addWorkerField(key,value) 
        
        self.workerMeasurement['measurement']='gpu_rig_'+ api_version
        self.workerMeasurement['tags']=self.workerTags
        self.workerMeasurement['fields']=self.workerFields           
    

        
    #def flatGpuField(self,data):    
    def extractGpus(self):
        tag_fields=['name','vendor','device_id','gpu_user_id']
        skip_fields = ['shares']
        if(self.login['success'] == 1):
            self.gpus=[]
            
            worker_name = self.summary['active_pool']['worker']
            
            for gpu in self.summary['gpus']:  
                gpuMeasurement={}
                gpuTags={}
                gpuFields={}
                gpuTags['worker']= worker_name
                
                for p in gpu:
                    if isinstance(p,dict):
                        pass
                    elif p in skip_fields:
                        pass
                    else:
                        if(p in tag_fields):
                            gpuTags[p] = gpu[p]
                        else:       
                            gpuFields[p] = gpu[p]
                            
                gpuMeasurement['measurement']='gpu_miner_'+ api_version
                gpuMeasurement['tags'] = gpuTags
                gpuMeasurement['fields'] = gpuFields            
                self.gpus.append(gpuMeasurement)
       
    
    def initial_connect(self):
        self.login_()
        self.updateStatus()
        self.workerTags['ip']=self.ip
        
    def updateStatus(self):
        self.reset_()
        self.getSummary()  
        self.extractWorker(self.summary)
        self.extractGpus()

In [ ]:
def insertInfluxDb(miner_list):
    try:
        with InfluxDBClient(url=url, token=token, org=org) as client:
                write_api = client.write_api(write_options=SYNCHRONOUS)               
                for miner in miner_list:
                    #print('insert: '+ miner.ip + ' '+str(miner.workerMeasurement))
                    try:
                        write_api.write(bucket, org, miner.workerMeasurement)
                        #print('completed')
                        for gpu in miner.gpus:
                            write_api.write(bucket, org, gpu)
                    except Exception as ex:
                        print (miner.ip)
                        #print (miner.stat_data)
                        print (ex)
                client.close()
    except Exception as e:
        print(e)
            

In [ ]:
def getMiners():
    miners =[]
    for ip in ips:
        miner = ethMiner(ip,password_)
        miner.login_()
        if(miner.login['success'] == 1):
            miners.append(miner)
            print(miner.ip)
    
    return miners   


In [ ]:
def updateMinerStat(miners_):
    for miner in miners_:
        miner.updateStatus()

In [ ]:
def main():    
    miners = getMiners()  
    print(len(miners))
    i = 0
    while 1:
        updateMinerStat(miners)        
        insertInfluxDb(miners)
        i = i+1
        print (i)
        time.sleep(60)
    return miners        

In [ ]:
main()